# 天池大赛·印象盐城

## 特征工程

初赛提供2012年1月-2017年10月盐城分车型销量配置数据。
第一阶段需要参赛者预测2017年11月盐城分车型销量数据，第二阶段需要参赛者预测2017年12月盐城分车型销量数据。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
train = pd.read_csv('../[new] yancheng_train_20171226.csv', low_memory=False)
test = pd.read_csv('../yancheng_testA_20171225.csv')

### 一、异常值处理

1.1 发现数据里相同车型在同一月份里有两条记录，这里做简单加和

In [3]:
labels = ['sale_date','class_id','brand_id','compartment','type_id','level_id','department_id','TR','gearbox_type','displacement','if_charging',
          'price_level','driven_type_id','fuel_type_id','newenergy_type_id','emission_standards_id','if_MPV_id','if_luxurious_id','power',
          'cylinder_number','engine_torque','car_length','car_width','car_height','total_quality','equipment_quality','rated_passenger',
          'wheelbase','front_track','rear_track']
train = train.groupby(labels).agg('sum').reset_index()

1.2 三个功率为81/70的条目里共有两款车

    其中一款车有记载的功率为66和81，我们令值为81
    
    另一款车有记载的功率为66，70,81和96，也令值为81

In [4]:
train.loc[12382,'power']=81
train.loc[13478,'power']=81
train.loc[13450,'power']=81
train[['power']] = train[['power']].astype('float32')

1.3 发动机扭矩

    有三个功率是'155/140'，就当是155吧
    
    还有一个车型是'-'，给个平均值吧

In [5]:
train.loc[12382,'engine_torque']=155
train.loc[13478,'engine_torque']=155
train.loc[13450,'engine_torque']=155
train['engine_torque'][train['engine_torque']=='-']=201.8
train['engine_torque'] = train['engine_torque'].astype('float32')

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### 二、特征处理

**训练数据**

In [6]:
labels = ['class_id','brand_id','compartment','type_id','level_id','department_id','TR','gearbox_type','displacement','if_charging',
          'price_level','driven_type_id','fuel_type_id','newenergy_type_id','emission_standards_id','if_MPV_id','if_luxurious_id','power',
          'cylinder_number','engine_torque','car_length','car_width','car_height','total_quality','equipment_quality','rated_passenger',
          'wheelbase','front_track','rear_track']
trainLabels = train[labels].drop_duplicates().reset_index(drop=True)
trainLabels.shape

(1539, 29)

In [7]:
trainDate = train
trainDate['year'] = trainDate['sale_date'].apply(lambda x: int(str(x)[0:4]))
trainDate['mouth'] = trainDate['sale_date'].apply(lambda x: int(str(x)[4:6]))
trainDate = trainDate.drop(['sale_date'], axis=1)

In [8]:
trainSaleDate = trainLabels

# 2012年1月的销量信息
trainTmp = trainDate[(trainDate['year']==2012)&(trainDate['mouth']==1)]
# trainTmp = trainTmp[trainTmp['mouth']==1]
trainTmp.loc[:, 'how_many_mouth_has_sell'] = 1
sale_class_count = len(trainTmp)

trainSaleDate = pd.merge(trainSaleDate, trainTmp, how='left')
trainSaleDate['year'].fillna(2012.0, inplace=True)
trainSaleDate['mouth'].fillna(1.0, inplace=True)
# trainSaleDate['how_many_mouth_has_sell'].fillna(0.0, inplace=True)
trainSaleDate.loc[:, 'sale_quantity_total_last_mouth'] = 0
trainSaleDate.loc[:, 'sale_quantity_this_last_mouth'] = 0
trainSaleDate.loc[:, 'this_mouth_sale_class_count'] = sale_class_count
# trainSaleDate

labels = ['class_id','sale_quantity','brand_id','compartment','type_id','level_id','department_id','TR','gearbox_type','displacement',
          'if_charging','price_level','driven_type_id','fuel_type_id','newenergy_type_id','emission_standards_id','if_MPV_id','if_luxurious_id',
          'power','cylinder_number','engine_torque','car_length','car_width','car_height','total_quality','equipment_quality','rated_passenger',
          'wheelbase','front_track','rear_track','how_many_mouth_has_sell']
labels_year = ['class_id','sale_quantity','brand_id','compartment','type_id','level_id','department_id','TR','gearbox_type','displacement','if_charging',
               'price_level','driven_type_id','fuel_type_id','newenergy_type_id','emission_standards_id','if_MPV_id','if_luxurious_id','power',
               'cylinder_number','engine_torque','car_length','car_width','car_height','total_quality','equipment_quality','rated_passenger',
               'wheelbase','front_track','rear_track','year','mouth']
for mouth in range(2, 13):
    trainSaleDateTmp = trainLabels
    trainTmp = trainDate[(trainDate['year']==2012)&(trainDate['mouth']==mouth)]
    
    sellMouth = trainSaleDate[labels][(trainSaleDate['year']==2012)&(trainSaleDate['mouth']==(mouth-1))]
    sellMouth.dropna(inplace=True)
    sellMouth['how_many_mouth_has_sell'] = sellMouth['how_many_mouth_has_sell']+1
    sale_quantity_total_last_mouth = sellMouth['sale_quantity'].sum()
    sellMouth.rename(columns={'sale_quantity': 'sale_quantity_this_last_mouth'}, inplace=True)
    
    trainTmp = pd.merge(trainTmp, sellMouth, how='left')
    trainTmp['how_many_mouth_has_sell'].fillna(1.0, inplace=True)
    sale_class_count = len(trainTmp)
    
    trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, how='left')
    trainSaleDateTmp['year'].fillna(2012.0, inplace=True)
    trainSaleDateTmp['mouth'].fillna(mouth, inplace=True)
    trainSaleDateTmp.loc[:, 'sale_quantity_total_last_mouth'] = sale_quantity_total_last_mouth
    trainSaleDateTmp.loc[:, 'this_mouth_sale_class_count'] = sale_class_count
    
    trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])
    
trainSaleDate.loc[:, 'sale_quantity_total_last_year'] = 0
trainSaleDate.loc[:, 'sale_quantity_this_last_year'] = 0

for year in range(2013, 2017):
    for mouth in range(1, 13):
        trainSaleDateTmp = trainLabels
        trainTmp = trainDate[(trainDate['year']==year)&(trainDate['mouth']==mouth)]
        
        if mouth == 1:
            sellMouth = trainSaleDate[labels][(trainSaleDate['year']==(year-1))&(trainSaleDate['mouth']==12)]
        else:
            sellMouth = trainSaleDate[labels][(trainSaleDate['year']==year)&(trainSaleDate['mouth']==(mouth-1))]
        sellMouth.dropna(inplace=True)
        sellMouth['how_many_mouth_has_sell'] = sellMouth['how_many_mouth_has_sell']+1
        sale_quantity_total_last_mouth = sellMouth['sale_quantity'].sum()
        sellMouth.rename(columns={'sale_quantity': 'sale_quantity_this_last_mouth'}, inplace=True)
        
        sellYear = trainSaleDate[labels_year][((trainSaleDate['year']==(year-1))&(trainSaleDate['mouth']>=mouth))|((trainSaleDate['year']==year)&(trainSaleDate['mouth']<mouth))]
        sellYear.dropna(inplace=True)
        sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
        sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
        sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)
        
        trainTmp = pd.merge(trainTmp, sellMouth, how='left')
        trainTmp['how_many_mouth_has_sell'].fillna(1.0, inplace=True)
        sale_class_count = len(trainTmp)
        
        trainTmp = pd.merge(trainTmp, sellYear, how='left')
        
        trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, how='left')
        trainSaleDateTmp['year'].fillna(year, inplace=True)
        trainSaleDateTmp['mouth'].fillna(mouth, inplace=True)
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_mouth'] = sale_quantity_total_last_mouth
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
        trainSaleDateTmp.loc[:, 'this_mouth_sale_class_count'] = sale_class_count
        
        trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])
        
        print('year: '+str(year)+', mouth: '+str(mouth)+'--'+str(trainSaleDate.shape[0]/1539))
        
for year in range(2017, 2018):
    for mouth in range(1, 11):
        trainSaleDateTmp = trainLabels
        trainTmp = trainDate[(trainDate['year']==year)&(trainDate['mouth']==mouth)]
        
        if mouth == 1:
            sellMouth = trainSaleDate[labels][(trainSaleDate['year']==(year-1))&(trainSaleDate['mouth']==12)]
        else:
            sellMouth = trainSaleDate[labels][(trainSaleDate['year']==year)&(trainSaleDate['mouth']==(mouth-1))]
        sellMouth.dropna(inplace=True)
        sellMouth['how_many_mouth_has_sell'] = sellMouth['how_many_mouth_has_sell']+1
        sale_quantity_total_last_mouth = sellMouth['sale_quantity'].sum()
        sellMouth.rename(columns={'sale_quantity': 'sale_quantity_this_last_mouth'}, inplace=True)
        
        sellYear = trainSaleDate[labels_year][((trainSaleDate['year']==(year-1))&(trainSaleDate['mouth']>=mouth))|((trainSaleDate['year']==year)&(trainSaleDate['mouth']<mouth))]
        sellYear.dropna(inplace=True)
        sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
        sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
        sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)
        
        trainTmp = pd.merge(trainTmp, sellMouth, how='left')
        trainTmp['how_many_mouth_has_sell'].fillna(1.0, inplace=True)
        sale_class_count = len(trainTmp)
        
        trainTmp = pd.merge(trainTmp, sellYear, how='left')
        
        trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, how='left')
        trainSaleDateTmp['year'].fillna(year, inplace=True)
        trainSaleDateTmp['mouth'].fillna(mouth, inplace=True)
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_mouth'] = sale_quantity_total_last_mouth
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
        trainSaleDateTmp.loc[:, 'this_mouth_sale_class_count'] = sale_class_count
        
        trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])
        
        print('year: '+str(year)+', mouth: '+str(mouth)+'--'+str(trainSaleDate.shape[0]/1539))
        
    for mouth in range(11, 12):
        trainSaleDateTmp = trainLabels
        
        sellMouth = trainSaleDate[labels][(trainSaleDate['year']==year)&(trainSaleDate['mouth']==(mouth-1))]
        sellMouth.dropna(inplace=True)
        sellMouth['how_many_mouth_has_sell'] = sellMouth['how_many_mouth_has_sell']+1
        sale_quantity_total_last_mouth = sellMouth['sale_quantity'].sum()
        sellMouth.rename(columns={'sale_quantity': 'sale_quantity_this_last_mouth'}, inplace=True)
        
        sellYear = trainSaleDate[labels_year][((trainSaleDate['year']==(year-1))&(trainSaleDate['mouth']>=mouth))|((trainSaleDate['year']==year)&(trainSaleDate['mouth']<mouth))]
        sellYear.dropna(inplace=True)
        sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
        sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
        sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)
        
        trainSaleDateTmp = pd.merge(trainSaleDateTmp, sellMouth, how='left')
        trainSaleDateTmp['how_many_mouth_has_sell'].fillna(1.0, inplace=True)
        sale_class_count = len(trainSaleDateTmp)
        
        trainSaleDateTmp = pd.merge(trainSaleDateTmp, sellYear, how='left')
        
        trainSaleDateTmp.loc[:, 'year'] = 2017
        trainSaleDateTmp.loc[:, 'mouth'] = 11
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_mouth'] = sale_quantity_total_last_mouth
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
        trainSaleDateTmp.loc[:, 'this_mouth_sale_class_count'] = sale_class_count
        trainSaleDateTmp.loc[:, 'sale_quantity'] = 0
        
        trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])
        
        print('year: '+str(year)+', mouth: '+str(mouth)+'--'+str(trainSaleDate.shape[0]/1539))
        
trainSaleDate.sort(['year', 'mouth', 'class_id'])
trainSaleDate.fillna(0.0, inplace=True)
# trainSaleDate.to_csv('trainSaleDate_train.csv', index=False)

C:\Users\yudake\Anaconda3\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\yudake\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


year: 2013, mouth: 1--13.0
year: 2013, mouth: 2--14.0
year: 2013, mouth: 3--15.0
year: 2013, mouth: 4--16.0
year: 2013, mouth: 5--17.0
year: 2013, mouth: 6--18.0
year: 2013, mouth: 7--19.0
year: 2013, mouth: 8--20.0
year: 2013, mouth: 9--21.0
year: 2013, mouth: 10--22.0
year: 2013, mouth: 11--23.0
year: 2013, mouth: 12--24.0
year: 2014, mouth: 1--25.0
year: 2014, mouth: 2--26.0
year: 2014, mouth: 3--27.0
year: 2014, mouth: 4--28.0
year: 2014, mouth: 5--29.0
year: 2014, mouth: 6--30.0
year: 2014, mouth: 7--31.0
year: 2014, mouth: 8--32.0
year: 2014, mouth: 9--33.0
year: 2014, mouth: 10--34.0
year: 2014, mouth: 11--35.0
year: 2014, mouth: 12--36.0
year: 2015, mouth: 1--37.0
year: 2015, mouth: 2--38.0
year: 2015, mouth: 3--39.0
year: 2015, mouth: 4--40.0
year: 2015, mouth: 5--41.0
year: 2015, mouth: 6--42.0
year: 2015, mouth: 7--43.0
year: 2015, mouth: 8--44.0
year: 2015, mouth: 9--45.0
year: 2015, mouth: 10--46.0
year: 2015, mouth: 11--47.0
year: 2015, mouth: 12--48.0
year: 2016, mouth: 

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:157: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


2.1 品牌ID共有36个，进行One-hot编码

In [9]:
brand_id_dummies = pd.get_dummies(trainSaleDate['brand_id'], prefix='brand_id')
trainSaleDate = pd.concat([trainSaleDate, brand_id_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['brand_id'], axis=1)
trainSaleDate.head()

,TR,car_height,car_length,car_width,class_id,compartment,cylinder_number,department_id,displacement,driven_type_id,...,brand_id_813,brand_id_814,brand_id_831,brand_id_836,brand_id_841,brand_id_864,brand_id_872,brand_id_923,brand_id_953,brand_id_985
0,6,1750,5213,1847,125403,2,4,2,2.4,1,...,0,0,0,0,0,0,0,0,0,0
1,6,1772,5256,1878,125403,2,4,2,2.4,1,...,0,0,0,0,0,0,0,0,0,0
2,6,1772,5256,1878,125403,2,6,2,3.0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,1485,4825,1820,136916,3,4,5,2.0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,1485,4825,1820,136916,3,4,5,2.0,1,...,0,0,0,0,0,0,0,0,0,0


2.2 厢数有三种取值，进行One-hot编码

In [10]:
compartment_dummies = pd.get_dummies(trainSaleDate['compartment'], prefix='compartment')
trainSaleDate = pd.concat([trainSaleDate, compartment_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['compartment'], axis=1)
trainSaleDate.head()

,TR,car_height,car_length,car_width,class_id,cylinder_number,department_id,displacement,driven_type_id,emission_standards_id,...,brand_id_836,brand_id_841,brand_id_864,brand_id_872,brand_id_923,brand_id_953,brand_id_985,compartment_1,compartment_2,compartment_3
0,6,1750,5213,1847,125403,4,2,2.4,1,3,...,0,0,0,0,0,0,0,0,1,0
1,6,1772,5256,1878,125403,4,2,2.4,1,3,...,0,0,0,0,0,0,0,0,1,0
2,6,1772,5256,1878,125403,6,2,3.0,1,3,...,0,0,0,0,0,0,0,0,1,0
3,4,1485,4825,1820,136916,4,5,2.0,1,3,...,0,0,0,0,0,0,0,0,0,1
4,4,1485,4825,1820,136916,4,5,2.0,1,3,...,0,0,0,0,0,0,0,0,0,1


2.3 车型类别有四种

In [11]:
type_id_dummies = pd.get_dummies(trainSaleDate['type_id'], prefix='type_id')
trainSaleDate = pd.concat([trainSaleDate, type_id_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['type_id'], axis=1)
trainSaleDate.head()

,TR,car_height,car_length,car_width,class_id,cylinder_number,department_id,displacement,driven_type_id,emission_standards_id,...,brand_id_923,brand_id_953,brand_id_985,compartment_1,compartment_2,compartment_3,type_id_1,type_id_2,type_id_3,type_id_4
0,6,1750,5213,1847,125403,4,2,2.4,1,3,...,0,0,0,0,1,0,0,0,1,0
1,6,1772,5256,1878,125403,4,2,2.4,1,3,...,0,0,0,0,1,0,0,0,1,0
2,6,1772,5256,1878,125403,6,2,3.0,1,3,...,0,0,0,0,1,0,0,0,1,0
3,4,1485,4825,1820,136916,4,5,2.0,1,3,...,0,0,0,0,0,1,0,1,0,0
4,4,1485,4825,1820,136916,4,5,2.0,1,3,...,0,0,0,0,0,1,0,1,0,0


2.4 车型级别有六种

In [12]:
level_id_dummies = pd.get_dummies(trainSaleDate['level_id'], prefix='level_id')
trainSaleDate = pd.concat([trainSaleDate, level_id_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['level_id'], axis=1)
trainSaleDate.head()

,TR,car_height,car_length,car_width,class_id,cylinder_number,department_id,displacement,driven_type_id,emission_standards_id,...,type_id_1,type_id_2,type_id_3,type_id_4,level_id_-,level_id_1,level_id_2,level_id_3,level_id_4,level_id_5
0,6,1750,5213,1847,125403,4,2,2.4,1,3,...,0,0,1,0,0,0,1,0,0,0
1,6,1772,5256,1878,125403,4,2,2.4,1,3,...,0,0,1,0,0,0,1,0,0,0
2,6,1772,5256,1878,125403,6,2,3.0,1,3,...,0,0,1,0,0,0,1,0,0,0
3,4,1485,4825,1820,136916,4,5,2.0,1,3,...,0,1,0,0,0,0,1,0,0,0
4,4,1485,4825,1820,136916,4,5,2.0,1,3,...,0,1,0,0,0,0,1,0,0,0


2.5 车型系别有七种

In [13]:
department_id_dummies = pd.get_dummies(trainSaleDate['department_id'], prefix='department_id')
trainSaleDate = pd.concat([trainSaleDate, department_id_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['department_id'], axis=1)
trainSaleDate.head()

,TR,car_height,car_length,car_width,class_id,cylinder_number,displacement,driven_type_id,emission_standards_id,engine_torque,...,level_id_3,level_id_4,level_id_5,department_id_1,department_id_2,department_id_3,department_id_4,department_id_5,department_id_6,department_id_7
0,6,1750,5213,1847,125403,4,2.4,1,3,225.0,...,0,0,0,0,1,0,0,0,0,0
1,6,1772,5256,1878,125403,4,2.4,1,3,225.0,...,0,0,0,0,1,0,0,0,0,0
2,6,1772,5256,1878,125403,6,3.0,1,3,290.0,...,0,0,0,0,1,0,0,0,0,0
3,4,1485,4825,1820,136916,4,2.0,1,3,190.0,...,0,0,0,0,0,0,0,1,0,0
4,4,1485,4825,1820,136916,4,2.0,1,3,190.0,...,0,0,0,0,0,0,0,1,0,0


2.6 变速器档位有十种

In [14]:
TR_dummies = pd.get_dummies(trainSaleDate['TR'], prefix='TR')
trainSaleDate = pd.concat([trainSaleDate, TR_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['TR'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,cylinder_number,displacement,driven_type_id,emission_standards_id,engine_torque,equipment_quality,...,TR_0,TR_1,TR_4,TR_5,TR_5;4,TR_6,TR_7,TR_8,TR_8;7,TR_9
0,1750,5213,1847,125403,4,2.4,1,3,225.0,1840,...,0,0,0,0,0,1,0,0,0,0
1,1772,5256,1878,125403,4,2.4,1,3,225.0,1840,...,0,0,0,0,0,1,0,0,0,0
2,1772,5256,1878,125403,6,3.0,1,3,290.0,1930,...,0,0,0,0,0,1,0,0,0,0
3,1485,4825,1820,136916,4,2.0,1,3,190.0,1490,...,0,0,1,0,0,0,0,0,0,0
4,1485,4825,1820,136916,4,2.0,1,3,190.0,1520,...,0,0,1,0,0,0,0,0,0,0


2.7 变速器形式有七种

In [15]:
gearbox_type_dummies = pd.get_dummies(trainSaleDate['gearbox_type'], prefix='gearbox_type')
trainSaleDate = pd.concat([trainSaleDate, gearbox_type_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['gearbox_type'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,cylinder_number,displacement,driven_type_id,emission_standards_id,engine_torque,equipment_quality,...,TR_8,TR_8;7,TR_9,gearbox_type_AMT,gearbox_type_AT,gearbox_type_AT;DCT,gearbox_type_CVT,gearbox_type_DCT,gearbox_type_MT,gearbox_type_MT;AT
0,1750,5213,1847,125403,4,2.4,1,3,225.0,1840,...,0,0,0,0,1,0,0,0,0,0
1,1772,5256,1878,125403,4,2.4,1,3,225.0,1840,...,0,0,0,0,1,0,0,0,0,0
2,1772,5256,1878,125403,6,3.0,1,3,290.0,1930,...,0,0,0,0,1,0,0,0,0,0
3,1485,4825,1820,136916,4,2.0,1,3,190.0,1490,...,0,0,0,0,1,0,0,0,0,0
4,1485,4825,1820,136916,4,2.0,1,3,190.0,1520,...,0,0,0,0,1,0,0,0,0,0


2.8 排量

    区间缩放

In [16]:
trainSaleDate.loc[:,'no_displacement']=0
trainSaleDate['no_displacement'][trainSaleDate['displacement']==0]=1
trainSaleDate['displacement'] = trainSaleDate['displacement'].apply(lambda x: (x-1.0)/(3.6-1.0))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,car_height,car_length,car_width,class_id,cylinder_number,displacement,driven_type_id,emission_standards_id,engine_torque,equipment_quality,...,TR_8;7,TR_9,gearbox_type_AMT,gearbox_type_AT,gearbox_type_AT;DCT,gearbox_type_CVT,gearbox_type_DCT,gearbox_type_MT,gearbox_type_MT;AT,no_displacement
0,1750,5213,1847,125403,4,0.538462,1,3,225.0,1840,...,0,0,0,1,0,0,0,0,0,0
1,1772,5256,1878,125403,4,0.538462,1,3,225.0,1840,...,0,0,0,1,0,0,0,0,0,0
2,1772,5256,1878,125403,6,0.769231,1,3,290.0,1930,...,0,0,0,1,0,0,0,0,0,0
3,1485,4825,1820,136916,4,0.384615,1,3,190.0,1490,...,0,0,0,1,0,0,0,0,0,0
4,1485,4825,1820,136916,4,0.384615,1,3,190.0,1520,...,0,0,0,1,0,0,0,0,0,0


2.9 是否增压

In [17]:
if_charging_dummies = pd.get_dummies(trainSaleDate['if_charging'], prefix='if_charging')
trainSaleDate = pd.concat([trainSaleDate, if_charging_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['if_charging'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,cylinder_number,displacement,driven_type_id,emission_standards_id,engine_torque,equipment_quality,...,gearbox_type_AMT,gearbox_type_AT,gearbox_type_AT;DCT,gearbox_type_CVT,gearbox_type_DCT,gearbox_type_MT,gearbox_type_MT;AT,no_displacement,if_charging_L,if_charging_T
0,1750,5213,1847,125403,4,0.538462,1,3,225.0,1840,...,0,1,0,0,0,0,0,0,1,0
1,1772,5256,1878,125403,4,0.538462,1,3,225.0,1840,...,0,1,0,0,0,0,0,0,1,0
2,1772,5256,1878,125403,6,0.769231,1,3,290.0,1930,...,0,1,0,0,0,0,0,0,1,0
3,1485,4825,1820,136916,4,0.384615,1,3,190.0,1490,...,0,1,0,0,0,0,0,0,1,0
4,1485,4825,1820,136916,4,0.384615,1,3,190.0,1520,...,0,1,0,0,0,0,0,0,1,0


2.10 成交段

In [18]:
price_level_dummies = pd.get_dummies(trainSaleDate['price_level'], prefix='price_level')
trainSaleDate = pd.concat([trainSaleDate, price_level_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['price_level'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,cylinder_number,displacement,driven_type_id,emission_standards_id,engine_torque,equipment_quality,...,if_charging_T,price_level_10-15W,price_level_15-20W,price_level_20-25W,price_level_25-35W,price_level_35-50W,price_level_5-8W,price_level_50-75W,price_level_5WL,price_level_8-10W
0,1750,5213,1847,125403,4,0.538462,1,3,225.0,1840,...,0,0,0,0,0,1,0,0,0,0
1,1772,5256,1878,125403,4,0.538462,1,3,225.0,1840,...,0,0,0,0,0,1,0,0,0,0
2,1772,5256,1878,125403,6,0.769231,1,3,290.0,1930,...,0,0,0,0,0,1,0,0,0,0
3,1485,4825,1820,136916,4,0.384615,1,3,190.0,1490,...,0,0,1,0,0,0,0,0,0,0
4,1485,4825,1820,136916,4,0.384615,1,3,190.0,1520,...,0,0,1,0,0,0,0,0,0,0


2.11 驱动形式

In [19]:
driven_type_id_dummies = pd.get_dummies(trainSaleDate['driven_type_id'], prefix='driven_type_id')
trainSaleDate = pd.concat([trainSaleDate, driven_type_id_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['driven_type_id'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,cylinder_number,displacement,emission_standards_id,engine_torque,equipment_quality,front_track,...,price_level_20-25W,price_level_25-35W,price_level_35-50W,price_level_5-8W,price_level_50-75W,price_level_5WL,price_level_8-10W,driven_type_id_1,driven_type_id_2,driven_type_id_3
0,1750,5213,1847,125403,4,0.538462,3,225.0,1840,1593,...,0,0,1,0,0,0,0,1,0,0
1,1772,5256,1878,125403,4,0.538462,3,225.0,1840,1593,...,0,0,1,0,0,0,0,1,0,0
2,1772,5256,1878,125403,6,0.769231,3,290.0,1930,1593,...,0,0,1,0,0,0,0,1,0,0
3,1485,4825,1820,136916,4,0.384615,3,190.0,1490,1575,...,0,0,0,0,0,0,0,1,0,0
4,1485,4825,1820,136916,4,0.384615,3,190.0,1520,1575,...,0,0,0,0,0,0,0,1,0,0


2.12 燃料种类

In [20]:
fuel_type_id_dummies = pd.get_dummies(trainSaleDate['fuel_type_id'], prefix='fuel_type_id')
trainSaleDate = pd.concat([trainSaleDate, fuel_type_id_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['fuel_type_id'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,cylinder_number,displacement,emission_standards_id,engine_torque,equipment_quality,front_track,...,price_level_5WL,price_level_8-10W,driven_type_id_1,driven_type_id_2,driven_type_id_3,fuel_type_id_-,fuel_type_id_1,fuel_type_id_2,fuel_type_id_3,fuel_type_id_4
0,1750,5213,1847,125403,4,0.538462,3,225.0,1840,1593,...,0,0,1,0,0,0,1,0,0,0
1,1772,5256,1878,125403,4,0.538462,3,225.0,1840,1593,...,0,0,1,0,0,0,1,0,0,0
2,1772,5256,1878,125403,6,0.769231,3,290.0,1930,1593,...,0,0,1,0,0,0,1,0,0,0
3,1485,4825,1820,136916,4,0.384615,3,190.0,1490,1575,...,0,0,1,0,0,0,1,0,0,0
4,1485,4825,1820,136916,4,0.384615,3,190.0,1520,1575,...,0,0,1,0,0,0,1,0,0,0


2.13 新能源类型

In [21]:
newenergy_type_id_dummies = pd.get_dummies(trainSaleDate['newenergy_type_id'], prefix='newenergy_type_id')
trainSaleDate = pd.concat([trainSaleDate, newenergy_type_id_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['newenergy_type_id'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,cylinder_number,displacement,emission_standards_id,engine_torque,equipment_quality,front_track,...,driven_type_id_3,fuel_type_id_-,fuel_type_id_1,fuel_type_id_2,fuel_type_id_3,fuel_type_id_4,newenergy_type_id_1,newenergy_type_id_2,newenergy_type_id_3,newenergy_type_id_4
0,1750,5213,1847,125403,4,0.538462,3,225.0,1840,1593,...,0,0,1,0,0,0,1,0,0,0
1,1772,5256,1878,125403,4,0.538462,3,225.0,1840,1593,...,0,0,1,0,0,0,1,0,0,0
2,1772,5256,1878,125403,6,0.769231,3,290.0,1930,1593,...,0,0,1,0,0,0,1,0,0,0
3,1485,4825,1820,136916,4,0.384615,3,190.0,1490,1575,...,0,0,1,0,0,0,1,0,0,0
4,1485,4825,1820,136916,4,0.384615,3,190.0,1520,1575,...,0,0,1,0,0,0,1,0,0,0


2.14 排放标准

In [22]:
emission_standards_id_dummies = pd.get_dummies(trainSaleDate['emission_standards_id'], prefix='emission_standards_id')
trainSaleDate = pd.concat([trainSaleDate, emission_standards_id_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['emission_standards_id'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,cylinder_number,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,...,fuel_type_id_3,fuel_type_id_4,newenergy_type_id_1,newenergy_type_id_2,newenergy_type_id_3,newenergy_type_id_4,emission_standards_id_1,emission_standards_id_2,emission_standards_id_3,emission_standards_id_5
0,1750,5213,1847,125403,4,0.538462,225.0,1840,1593,1.0,...,0,0,1,0,0,0,0,0,1,0
1,1772,5256,1878,125403,4,0.538462,225.0,1840,1593,1.0,...,0,0,1,0,0,0,0,0,1,0
2,1772,5256,1878,125403,6,0.769231,290.0,1930,1593,1.0,...,0,0,1,0,0,0,0,0,1,0
3,1485,4825,1820,136916,4,0.384615,190.0,1490,1575,1.0,...,0,0,1,0,0,0,0,0,1,0
4,1485,4825,1820,136916,4,0.384615,190.0,1520,1575,1.0,...,0,0,1,0,0,0,0,0,1,0


2.15 是否微客MPV

In [23]:
if_MPV_id_dummies = pd.get_dummies(trainSaleDate['if_MPV_id'], prefix='if_MPV_id')
trainSaleDate = pd.concat([trainSaleDate, if_MPV_id_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['if_MPV_id'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,cylinder_number,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,...,newenergy_type_id_1,newenergy_type_id_2,newenergy_type_id_3,newenergy_type_id_4,emission_standards_id_1,emission_standards_id_2,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2
0,1750,5213,1847,125403,4,0.538462,225.0,1840,1593,1.0,...,1,0,0,0,0,0,1,0,0,1
1,1772,5256,1878,125403,4,0.538462,225.0,1840,1593,1.0,...,1,0,0,0,0,0,1,0,0,1
2,1772,5256,1878,125403,6,0.769231,290.0,1930,1593,1.0,...,1,0,0,0,0,0,1,0,0,1
3,1485,4825,1820,136916,4,0.384615,190.0,1490,1575,1.0,...,1,0,0,0,0,0,1,0,0,1
4,1485,4825,1820,136916,4,0.384615,190.0,1520,1575,1.0,...,1,0,0,0,0,0,1,0,0,1


2.16 是否豪华

In [24]:
if_luxurious_id_dummies = pd.get_dummies(trainSaleDate['if_luxurious_id'], prefix='if_luxurious_id')
trainSaleDate = pd.concat([trainSaleDate, if_luxurious_id_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['if_luxurious_id'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,cylinder_number,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,...,newenergy_type_id_3,newenergy_type_id_4,emission_standards_id_1,emission_standards_id_2,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2
0,1750,5213,1847,125403,4,0.538462,225.0,1840,1593,1.0,...,0,0,0,0,1,0,0,1,1,0
1,1772,5256,1878,125403,4,0.538462,225.0,1840,1593,1.0,...,0,0,0,0,1,0,0,1,1,0
2,1772,5256,1878,125403,6,0.769231,290.0,1930,1593,1.0,...,0,0,0,0,1,0,0,1,1,0
3,1485,4825,1820,136916,4,0.384615,190.0,1490,1575,1.0,...,0,0,0,0,1,0,0,1,1,0
4,1485,4825,1820,136916,4,0.384615,190.0,1520,1575,1.0,...,0,0,0,0,1,0,0,1,1,0


2.17 功率

    区间缩放

In [25]:
trainSaleDate['power'] = MinMaxScaler().fit_transform(trainSaleDate['power'].reshape(-1, 1))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':


,car_height,car_length,car_width,class_id,cylinder_number,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,...,newenergy_type_id_3,newenergy_type_id_4,emission_standards_id_1,emission_standards_id_2,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2
0,1750,5213,1847,125403,4,0.538462,225.0,1840,1593,1.0,...,0,0,0,0,1,0,0,1,1,0
1,1772,5256,1878,125403,4,0.538462,225.0,1840,1593,1.0,...,0,0,0,0,1,0,0,1,1,0
2,1772,5256,1878,125403,6,0.769231,290.0,1930,1593,1.0,...,0,0,0,0,1,0,0,1,1,0
3,1485,4825,1820,136916,4,0.384615,190.0,1490,1575,1.0,...,0,0,0,0,1,0,0,1,1,0
4,1485,4825,1820,136916,4,0.384615,190.0,1520,1575,1.0,...,0,0,0,0,1,0,0,1,1,0


2.17 缸数

In [26]:
cylinder_number_dummies = pd.get_dummies(trainSaleDate['cylinder_number'], prefix='cylinder_number')
trainSaleDate = pd.concat([trainSaleDate, cylinder_number_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['cylinder_number'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,1750,5213,1847,125403,0.538462,225.0,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
1,1772,5256,1878,125403,0.538462,225.0,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
2,1772,5256,1878,125403,0.769231,290.0,1930,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,0,1
3,1485,4825,1820,136916,0.384615,190.0,1490,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
4,1485,4825,1820,136916,0.384615,190.0,1520,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0


2.18 发动机扭矩

    区间缩放

In [27]:
trainSaleDate['engine_torque'] = MinMaxScaler().fit_transform(trainSaleDate['engine_torque'].reshape(-1, 1))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':


,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,1750,5213,1847,125403,0.538462,0.394366,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
1,1772,5256,1878,125403,0.538462,0.394366,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
2,1772,5256,1878,125403,0.769231,0.577465,1930,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,0,1
3,1485,4825,1820,136916,0.384615,0.295775,1490,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
4,1485,4825,1820,136916,0.384615,0.295775,1520,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0


2.19 车长

    区间缩放

In [28]:
trainSaleDate['car_length'] = MinMaxScaler().fit_transform(trainSaleDate['car_length'].reshape(-1, 1))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,1750,0.966688,1847,125403,0.538462,0.394366,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
1,1772,0.993715,1878,125403,0.538462,0.394366,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
2,1772,0.993715,1878,125403,0.769231,0.577465,1930,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,0,1
3,1485,0.722816,1820,136916,0.384615,0.295775,1490,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
4,1485,0.722816,1820,136916,0.384615,0.295775,1520,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0


2.20 车宽

    区间缩放

In [29]:
trainSaleDate['car_width'] = MinMaxScaler().fit_transform(trainSaleDate['car_width'].reshape(-1, 1))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,1750,0.966688,0.780093,125403,0.538462,0.394366,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
1,1772,0.993715,0.851852,125403,0.538462,0.394366,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
2,1772,0.993715,0.851852,125403,0.769231,0.577465,1930,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,0,1
3,1485,0.722816,0.717593,136916,0.384615,0.295775,1490,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
4,1485,0.722816,0.717593,136916,0.384615,0.295775,1520,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0


2.21 车高

    区间缩放

In [30]:
trainSaleDate['car_height'] = MinMaxScaler().fit_transform(trainSaleDate['car_height'].reshape(-1, 1))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,0.614679,0.966688,0.780093,125403,0.538462,0.394366,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
1,0.655046,0.993715,0.851852,125403,0.538462,0.394366,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
2,0.655046,0.993715,0.851852,125403,0.769231,0.577465,1930,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,0,1
3,0.128440,0.722816,0.717593,136916,0.384615,0.295775,1490,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
4,0.128440,0.722816,0.717593,136916,0.384615,0.295775,1520,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0


2.22 质量

    区间缩放

In [31]:
trainSaleDate['total_quality'] = MinMaxScaler().fit_transform(trainSaleDate['total_quality'].reshape(-1, 1))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,0.614679,0.966688,0.780093,125403,0.538462,0.394366,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
1,0.655046,0.993715,0.851852,125403,0.538462,0.394366,1840,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
2,0.655046,0.993715,0.851852,125403,0.769231,0.577465,1930,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,0,1
3,0.128440,0.722816,0.717593,136916,0.384615,0.295775,1490,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
4,0.128440,0.722816,0.717593,136916,0.384615,0.295775,1520,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0


2.23 装备质量

    区间缩放

In [32]:
trainSaleDate['equipment_quality'] = MinMaxScaler().fit_transform(trainSaleDate['equipment_quality'].reshape(-1, 1))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,0.614679,0.966688,0.780093,125403,0.538462,0.394366,0.847222,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
1,0.655046,0.993715,0.851852,125403,0.538462,0.394366,0.847222,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
2,0.655046,0.993715,0.851852,125403,0.769231,0.577465,0.930556,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,0,1
3,0.128440,0.722816,0.717593,136916,0.384615,0.295775,0.523148,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
4,0.128440,0.722816,0.717593,136916,0.384615,0.295775,0.550926,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0


2.24 轴距

    区间缩放

In [33]:
trainSaleDate['wheelbase'] = MinMaxScaler().fit_transform(trainSaleDate['wheelbase'].reshape(-1, 1))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,0.614679,0.966688,0.780093,125403,0.538462,0.394366,0.847222,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
1,0.655046,0.993715,0.851852,125403,0.538462,0.394366,0.847222,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
2,0.655046,0.993715,0.851852,125403,0.769231,0.577465,0.930556,1593,1.0,1.0,...,1,0,0,1,1,0,0,0,0,1
3,0.128440,0.722816,0.717593,136916,0.384615,0.295775,0.523148,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
4,0.128440,0.722816,0.717593,136916,0.384615,0.295775,0.550926,1575,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0


2.25 前轮距

    区间缩放

In [34]:
trainSaleDate['front_track'] = MinMaxScaler().fit_transform(trainSaleDate['front_track'].reshape(-1, 1))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,0.614679,0.966688,0.780093,125403,0.538462,0.394366,0.847222,0.834667,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
1,0.655046,0.993715,0.851852,125403,0.538462,0.394366,0.847222,0.834667,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
2,0.655046,0.993715,0.851852,125403,0.769231,0.577465,0.930556,0.834667,1.0,1.0,...,1,0,0,1,1,0,0,0,0,1
3,0.128440,0.722816,0.717593,136916,0.384615,0.295775,0.523148,0.786667,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
4,0.128440,0.722816,0.717593,136916,0.384615,0.295775,0.550926,0.786667,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0


2.26 后轮距

    区间缩放

In [35]:
trainSaleDate['rear_track'] = MinMaxScaler().fit_transform(trainSaleDate['rear_track'].reshape(-1, 1))
trainSaleDate.head()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,0.614679,0.966688,0.780093,125403,0.538462,0.394366,0.847222,0.834667,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
1,0.655046,0.993715,0.851852,125403,0.538462,0.394366,0.847222,0.834667,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
2,0.655046,0.993715,0.851852,125403,0.769231,0.577465,0.930556,0.834667,1.0,1.0,...,1,0,0,1,1,0,0,0,0,1
3,0.128440,0.722816,0.717593,136916,0.384615,0.295775,0.523148,0.786667,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0
4,0.128440,0.722816,0.717593,136916,0.384615,0.295775,0.550926,0.786667,1.0,1.0,...,1,0,0,1,1,0,0,0,1,0


2.27 额定载客

In [36]:
rated_passenger_dummies = pd.get_dummies(trainSaleDate['rated_passenger'], prefix='rated_passenger')
trainSaleDate = pd.concat([trainSaleDate, rated_passenger_dummies], axis=1)
trainSaleDate = trainSaleDate.drop(['rated_passenger'], axis=1)
trainSaleDate.head()

,car_height,car_length,car_width,class_id,displacement,engine_torque,equipment_quality,front_track,how_many_mouth_has_sell,mouth,...,rated_passenger_4,rated_passenger_4-5,rated_passenger_5,rated_passenger_5-7,rated_passenger_5-8,rated_passenger_6-7,rated_passenger_6-8,rated_passenger_7,rated_passenger_7-8,rated_passenger_9
0,0.614679,0.966688,0.780093,125403,0.538462,0.394366,0.847222,0.834667,1.0,1.0,...,0,0,0,0,0,0,0,1,0,0
1,0.655046,0.993715,0.851852,125403,0.538462,0.394366,0.847222,0.834667,1.0,1.0,...,0,0,0,0,0,0,0,1,0,0
2,0.655046,0.993715,0.851852,125403,0.769231,0.577465,0.930556,0.834667,1.0,1.0,...,0,0,0,0,0,0,0,1,0,0
3,0.128440,0.722816,0.717593,136916,0.384615,0.295775,0.523148,0.786667,1.0,1.0,...,0,0,1,0,0,0,0,0,0,0
4,0.128440,0.722816,0.717593,136916,0.384615,0.295775,0.550926,0.786667,1.0,1.0,...,0,0,1,0,0,0,0,0,0,0


### 三、写入文件

In [37]:
trainSaleDate.shape

(109269, 140)

In [38]:
trainSaleDate.to_csv('train_feature.csv', index=False)